In [24]:
import os
import polars as pl

# Import Dataset

In [68]:
local_path='dataset/soccer18-19.csv'
auto='auto'

In [71]:
# get current directory
path = os.getcwd()
print("Current Directory", path)
print()
 
# parent directory
parent = os.path.dirname(path)
print("Parent directory", parent)
print()

# dataset path
dataset_path=os.path.join(parent,local_path)
print("Dataset path",dataset_path)
print()

# auto path
path_auto=os.path.join(parent,auto)
print("Dump path",path_auto)

Current Directory /home/baptiste/GDrive/learning/Python/polars_explo/project/football/notebook

Parent directory /home/baptiste/GDrive/learning/Python/polars_explo/project/football

Dataset path /home/baptiste/GDrive/learning/Python/polars_explo/project/football/dataset/soccer18-19.csv

Dump path /home/baptiste/GDrive/learning/Python/polars_explo/project/football/auto


In [72]:
df=polars.read_csv(dataset_path)

print("shape:", df.shape)

df.head(5)

shape: (380, 23)


Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
str,str,str,str,i64,i64,str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""E0""","""10/8/18""","""Man United""","""Leicester""",2,1,"""H""",1,0,"""H""","""A Marriner""",8,13,6,4,11,8,2,5,2,1,0,0
"""E0""","""11/8/18""","""Bournemouth""","""Cardiff""",2,0,"""H""",1,0,"""H""","""K Friend""",12,10,4,1,11,9,7,4,1,1,0,0
"""E0""","""11/8/18""","""Fulham""","""Crystal Palace…",0,2,"""A""",0,1,"""A""","""M Dean""",15,10,6,9,9,11,5,5,1,2,0,0
"""E0""","""11/8/18""","""Huddersfield""","""Chelsea""",0,3,"""A""",0,2,"""A""","""C Kavanagh""",6,13,1,4,9,8,2,5,2,1,0,0
"""E0""","""11/8/18""","""Newcastle""","""Tottenham""",1,2,"""A""",1,2,"""A""","""M Atkinson""",15,15,2,5,11,12,3,5,2,2,0,0


# Challenges

 ## Maximum number of fouls in one game

In [73]:
#max_foul=df.select(pl.max("HF","AF")).select(pl.max("max"))[0]

max_foul=max(df['HF'].max(),df['AF'].max())

print('Maximum Fouls in one game:',max_foul)

Maximum Fouls in one game: 23


## Game in which maximum number of fouls happened

In [74]:
df.filter((df['HF'] == max_foul)|(df['AF'] == max_foul))

Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
str,str,str,str,i64,i64,str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""E0""","""20/10/2018""","""Wolves""","""Watford""",0,2,"""A""",0,2,"""A""","""L Mason""",10,9,1,3,23,13,8,2,3,1,0,0


## Team committing the highest number of fools in average

### Shape the data games per team

In [106]:

# Home team reshape
col_home=['Div','HomeTeam','FTHG','HTHG','HS','HST','HF','HC','HY','HR','Score']
rename_col_home={
    'Div':'Div',
    'HomeTeam':'Team',
    'FTHG':'FTG',
    'HTHG':'HTG',
    'HS':'S',
    'HST':'ST',
    'HF':'F',
    'HC':'C',
    'HY':'Y',
    'HR':'R'}

df_home=df.with_columns(
    pl.when(pl.col('FTR')=='H').then('WIN')
    .otherwise((pl.when(pl.col('FTR')=='A').then('LOOSE')).otherwise('DRAW'))
    .alias('Score')
    ).select(col_home).rename(rename_col_home) 

# Away Team reshape
col_away=['Div','AwayTeam','FTAG','HTAG','AS','AST','AF','AC','AY','AR','Score']
rename_col_away={
    'Div':'Div',
    'AwayTeam':'Team',
    'FTAG':'FTG',
    'HTAG':'HTG',
    'AS':'S',
    'AST':'ST',
    'AF':'F',
    'AC':'C',
    'AY':'Y',
    'AR':'R'}

df_away=df.with_columns(
    pl.when(pl.col('FTR')=='H').then('LOOSE')
    .otherwise((pl.when(pl.col('FTR')=='A').then('LOOSE')).otherwise('DRAW'))
    .alias('Score')
    ).select(col_away).rename(rename_col_away) 

# Aggregate
df2=pl.concat([df_home,df_away])


df2.write_csv(os.path.join(path_auto,'df2.csv'))

print ('Shape:',df2.shape)
df2.head(8)

Shape: (760, 11)


Div,Team,FTG,HTG,S,ST,F,C,Y,R,Score
str,str,i64,i64,i64,i64,i64,i64,i64,i64,str
"""E0""","""Man United""",2,1,8,6,11,2,2,0,"""WIN"""
"""E0""","""Bournemouth""",2,1,12,4,11,7,1,0,"""WIN"""
"""E0""","""Fulham""",0,0,15,6,9,5,1,0,"""LOOSE"""
"""E0""","""Huddersfield""",0,0,6,1,9,2,2,0,"""LOOSE"""
"""E0""","""Newcastle""",1,1,15,2,11,3,2,0,"""LOOSE"""
"""E0""","""Watford""",2,1,19,5,10,8,2,0,"""WIN"""
"""E0""","""Wolves""",2,1,11,4,8,3,0,0,"""DRAW"""
"""E0""","""Arsenal""",0,0,9,3,11,2,2,0,"""LOOSE"""


### Group per team, concentrate on fouls

In [107]:
df3=df2.groupby('Team').agg(
    pl.col('F').mean().alias('F_mean'),
    pl.col('F').min().alias('F_min'),
    pl.col('F').max().alias('F_max'),
    pl.col('F').std().alias('F_std')
).sort('F_mean')

df3.write_csv(os.path.join(path_auto,'df3.csv'))

df3.head(30)

Team,F_mean,F_min,F_max,F_std
str,f64,i64,i64,f64
"""Liverpool""",8.289474,3,17,3.819636
"""Man City""",8.631579,2,14,3.10554
"""West Ham""",8.710526,2,14,2.577532
"""Chelsea""",8.921053,5,15,2.529625
"""Bournemouth""",8.921053,3,14,2.644541
"""Leicester""",9.236842,4,17,3.088201
"""Burnley""",9.447368,4,17,3.099236
"""Tottenham""",9.868421,3,19,3.78597
"""Cardiff""",10.105263,3,17,3.615204


## Percentage of Home win vs Away Win vs Draw

In [113]:
df4=df.groupby('FTR').agg(
    pl.col('Div').count().alias('Number')
)

df4.write_csv(os.path.join(path_auto,'df4.csv'))


df4

FTR,Number
str,u32
"""A""",128
"""H""",181
"""D""",71


# Impact of Red cards on the win probablity

In [143]:
df5 = df2.with_columns(
    pl.when(pl.col('Score')=='WIN').then(1) .otherwise(0).alias('Win'),
    pl.when(pl.col('Score')=='LOOSE').then(1) .otherwise(0).alias('Loose'),
    pl.when(pl.col('Score')=='DRAW').then(1) .otherwise(0).alias('Draw'),
)

df6=df5.groupby('R').agg(
    pl.col('Div').count().alias('NumberGame'),
    pl.col('Win').sum().alias('NumberWin'),
    pl.col('Loose').sum().alias('NumberDraw'),
    pl.col('Draw').sum().alias('NumberLoose'),
    (pl.col('Win').sum()/pl.col('Div').count()).alias('PercentWin'),
    (pl.col('Loose').sum()/pl.col('Div').count()).alias('PercentDraw'),
    (pl.col('Draw').sum()/pl.col('Div').count()).alias('PercentLoose')
)

df6.head

<bound method DataFrame.head of shape: (3, 8)
┌─────┬────────────┬───────────┬────────────┬─────────────┬────────────┬─────────────┬────────────┐
│ R   ┆ NumberGame ┆ NumberWin ┆ NumberDraw ┆ NumberLoose ┆ PercentWin ┆ PercentDraw ┆ PercentLoo │
│ --- ┆ ---        ┆ ---       ┆ ---        ┆ ---         ┆ ---        ┆ ---         ┆ se         │
│ i64 ┆ u32        ┆ i32       ┆ i32        ┆ i32         ┆ f64        ┆ f64         ┆ ---        │
│     ┆            ┆           ┆            ┆             ┆            ┆             ┆ f64        │
╞═════╪════════════╪═══════════╪════════════╪═════════════╪════════════╪═════════════╪════════════╡
│ 0   ┆ 714        ┆ 177       ┆ 404        ┆ 133         ┆ 0.247899   ┆ 0.565826    ┆ 0.186275   │
│ 1   ┆ 45         ┆ 4         ┆ 32         ┆ 9           ┆ 0.088889   ┆ 0.711111    ┆ 0.2        │
│ 2   ┆ 1          ┆ 0         ┆ 1          ┆ 0           ┆ 0.0        ┆ 1.0         ┆ 0.0        │
└─────┴────────────┴───────────┴────────────┴─────────